In [ ]:
import json
import numpy as np
import pandas as pd
from utility import *

homedir = get_homedir()
FIPS_mapping, FIPS_full = get_FIPS(reduced=True)

## Read Dataset

In [ ]:
with open(f'{homedir}/JK/po_code_state_map.json') as f:
    po_st = json.load(f)

st_to_fips = {}
po_to_fips = {}
for dic in po_st:
    st_to_fips[dic['state']] = dic['fips']
    po_to_fips[dic['postalCode']] = dic['fips']
print(st_to_fips['California'], po_to_fips['CA'])

In [ ]:
fips_key = pd.read_csv(f'{homedir}/data/us/processing_data/fips_key.csv', encoding='latin-1')
fips_key['FIPS'] = fips_key['FIPS'].apply(correct_FIPS)
fips_key.replace({'ST':po_to_fips}, inplace=True)
fips_key.drop_duplicates(subset=['FIPS'], inplace=True)
fips_key.drop(index=[2936, 2941], inplace=True)         ## Missing in demo & GDP
fips_key.head()

In [ ]:
GDP = pd.read_csv(f'{homedir}/JK/lagdp1219.csv')
GDP.replace({'state':st_to_fips}, inplace=True)
GDP.dropna(thresh=6, inplace=True)
GDP.head()

In [ ]:
STlist = list(GDP['state'].unique())
df = pd.DataFrame(columns=GDP.columns)
for state in STlist:
    fips_state = fips_key[fips_key['ST']==state].sort_values(by=['COUNTY'])
    GDP_state = GDP[GDP['state']==state].copy()
    if (len(GDP_state)==len(fips_state)):
        GDP_state['fips'] = list(fips_state['FIPS'])
        df = df.append(GDP_state)
    elif state=='08':
        df = df.append(GDP_state[~GDP_state['fips'].isna()])
        temp = GDP_state[GDP_state['fips'].isna()]
        print(len(temp), len(fips_state))
        temp['fips'] = list(fips_state['FIPS'])
        df = df.append(temp)
    else:
        df = df.append(GDP_state)
        # print('Length not match on', state)
        # print(len(GDP_state),len(fips_state))
        # for i in range(min(len(fips_state),len(GDP_state))):
        #     print(list(GDP_state['county'])[i],'-', list(fips_state['COUNTY'])[i])
df.head()

In [ ]:
df['fips'] = df['fips'].apply(correct_FIPS)
df[['fips','2015','2016','2017','2018']].to_csv('GDP.csv', index=False)